In [1]:
# Import necessary libraries
from qiskit import QuantumCircuit, Aer
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliSumOp
import numpy as np




In [2]:
# Define the gates for the Abelian calculation
# Here we use the Pauli-X and Pauli-Z gates as an example
X = PauliSumOp.from_list([('X', 0, 1)])
Z = PauliSumOp.from_list([('Z', 0, 1)])

# Define the Abelian group using the gates
group = [X, Z]

# Define the linear operator as a matrix
linear_matrix = np.array([[1, 1j], [1j, 1]]) / np.sqrt(2)# np.array([[1, -1j], [-1j, 1]]) / np.sqrt(2)

# Check if the matrix is unitary
if not np.allclose(np.eye(2), np.dot(linear_matrix, linear_matrix.conj().T)):
    raise ValueError("The linear matrix is not unitary.")



In [3]:

# Define the lattice as a list of tuples with the qubit indices for each coupling
lattice = [(0, 1)]

# Define the Hamiltonian as a sum of Pauli products for each coupling in the lattice
hamiltonian = 0
for coupling in lattice:
    hamiltonian += -1 * (X.tensor(Z) + Z.tensor(X)).to_pauli_op()

In [4]:

# Define the ansatz for the VQE algorithm
ansatz = EfficientSU2(num_qubits=4, reps=1)

# Define the optimizer for the VQE algorithm
optimizer = SLSQP()

# Define the quantum instance to use for the VQE algorithm
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))

# Define the VQE algorithm with the ansatz, optimizer, and quantum instance
vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=quantum_instance)

# Calculate the ground state energy using the VQE algorithm
ground_state_energy = vqe.compute_minimum_eigenvalue(hamiltonian)

# Print the ground state energy
print({'11 11 01': 502, '01 11 01': 498})
print("Ground state energy:", ground_state_energy)

{'11 11 01': 502, '01 11 01': 498}
Ground state energy: {   'aux_operator_eigenvalues': None,
    'cost_function_evals': 9,
    'eigenstate': array([ 0.25783356-0.35238118j,  0.43203725+0.07661926j,
        0.63652489-0.25409297j, -0.31982185+0.21168248j]),
    'eigenvalue': 0j,
    'optimal_circuit': None,
    'optimal_parameters': {   ParameterVectorElement(θ[0]): 2.1526440041184323,
                              ParameterVectorElement(θ[1]): -2.6144557872796996,
                              ParameterVectorElement(θ[2]): -1.7172431224061533,
                              ParameterVectorElement(θ[3]): -0.5781771093852943,
                              ParameterVectorElement(θ[4]): 2.284083475685806,
                              ParameterVectorElement(θ[5]): 2.6732514484990277,
                              ParameterVectorElement(θ[6]): -0.2570117759152253,
                              ParameterVectorElement(θ[7]): 1.3743647797522458},
    'optimal_point': array([ 2.152644  , -2.614

In [5]:
# Import necessary libraries
from qiskit import QuantumCircuit, Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info.operators import Operator


# Define the gates for the Abelian calculation
# Here we use the Pauli-X and Pauli-Z gates as an example
X = Operator.from_label('X')
Z = Operator.from_label('Z')

# Define the Abelian group using the gates
group = [X, Z]

# Define the linear operator as a matrix
linear_operator = np.array([[1, 1j], [1j, 1]]) / np.sqrt(2)

# Define the circuit with two qubits
qc = QuantumCircuit(4)

# Apply the group gates to the first qubit
for gate in group:
    qc.unitary(gate, [0])

# Apply the linear operator to the second qubit
qc.unitary(linear_operator, [1])

# Measure both qubits
qc.measure_all()

# Execute the circuit using the QASM simulator
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1000)
result = job.result()

# Print the measurement results
print(result.get_counts(qc))

{'0011': 480, '0001': 520}
